In [1]:
from structured_crs_report_analysis_burr import structured_csr_report_build, analysis_types
from structured_wikipedia_report_analysis_wikipedia_burr import structured_wikipedia_report_build
from uuid import uuid4
import lancedb
from datetime import datetime
import sqlite3
from pathlib import Path
from sentence_transformers import SentenceTransformer

/opt/anaconda3/envs/wonk/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
<All keys matched successfully>
<All keys matched successfully>


In [2]:
run_timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
run_id = str(uuid4())
# project_id = str(uuid4())
project_id = 'rural broadband'
crs_source_dataset = 'CRSReports'
wikipedia_source_dataset = 'Wikipedia'
subject_matter = "Rural Broadband in America."
focus = "Historical, current, and future challenges for rural broadband and connectivity in America."
analysis_type = 'domestic_policy'
analysis_prompts = analysis_types[analysis_type]

project_folder = Path('project_research')
project_folder.mkdir(parents=True, exist_ok=True)
research_json_folder = project_folder.joinpath('json_data')
research_json_folder.mkdir(parents=True, exist_ok=True)
database_location = project_folder.joinpath('research.sqlite')

conn = sqlite3.connect(database_location)
cursor = conn.cursor()
index = lancedb.connect('../wonky_data/indexes/')
table = index.open_table('sections_hybrid')
encoder = SentenceTransformer('nomic-ai/nomic-embed-text-v1.5', device='mps', trust_remote_code=True)

temperature = 0.2
crs_num_results = 3
wikipedia_num_results = 2

<All keys matched successfully>


ui_folder = Path('working_folder/nicegui')
ui_database_location = ui_folder.joinpath('ui_data.sqlite')
ui_conn = sqlite3.connect(ui_database_location)
ui_cursor = ui_conn.cursor()

In [78]:
crs_extraction_app = structured_csr_report_build()

In [79]:
crs_extraction_action, crs_extraction_result, crs_extraction_state = crs_extraction_app.run(
    halt_after=["save_research_to_json"],
    inputs={
        "subject_matter": subject_matter,
        "focus": focus,
        "analysis_type": analysis_type,
        "analysis_prompts": analysis_prompts,
        "table": table,
        "temperature":temperature,
        "run_id": run_id,
        "project_id": project_id,
        "run_timestamp": run_timestamp,
        "conn": conn,
        "cursor": cursor,
        "research_json_folder": research_json_folder,
        "num_results": crs_num_results,
        "source_dataset": crs_source_dataset
    }
)

['RL33816' 'R47017']


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

RL33816
RL33816__5___1
RL33816__6___1
RL33816__7___1
RL33816__8___1
RL33816__10___1
RL33816__16___1
RL33816__17___1
RL33816__18___1
RL33816__21___1
RL33816__24___1
10
R47017
R47017__1___1
R47017__2___1
R47017__3___1
R47017__4___1
R47017__5___1
R47017__7___1
R47017__8___1
R47017__9___1
R47017__11___1
R47017__16___1
R47017__17___1
R47017__18___1
R47017__22___1
R47017__23___1
14


In [97]:
crs_data = crs_extraction_state.get_all()

In [81]:
wikipedia_extraction_app = structured_wikipedia_report_build()

In [82]:
wikipedia_extraction_action, wikipedia_extraction_result, wikipedia_extraction_state = wikipedia_extraction_app.run(
    halt_after=["save_research_to_json"],
    inputs={
        "subject_matter": subject_matter,
        "focus": focus,
        "analysis_type": analysis_type,
        "analysis_prompts": analysis_prompts,
        "table": table,
        "temperature":temperature,
        "run_id": run_id,
        "project_id": project_id,
        "run_timestamp": run_timestamp,
        "conn": conn,
        "cursor": cursor,
        "research_json_folder": research_json_folder,
        "num_results": wikipedia_num_results,
        "source_dataset": wikipedia_source_dataset
    }
)

['Universal Service Fund', 'Infrastructure Investment and Jobs Act', 'Internet in the United States', 'Rural Utilities Service']


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

WIKI49281
4
dict_keys(['title', 'filename', 'number', 'source_file', 'typeId', 'sections', 'full_text', 'summary', 'citation', 'id', 'type', 'doc_id', 'source_document', 'source_dataset', 'topics', 'active', 'date', 'text'])
<class 'pandas.core.frame.DataFrame'>
                    title                                           filename  \
0  Universal Service Fund  https://en.wikipedia.org/wiki/Universal_Servic...   
1  Universal Service Fund  https://en.wikipedia.org/wiki/Universal_Servic...   
2  Universal Service Fund  https://en.wikipedia.org/wiki/Universal_Servic...   
3  Universal Service Fund  https://en.wikipedia.org/wiki/Universal_Servic...   
4  Universal Service Fund  https://en.wikipedia.org/wiki/Universal_Servic...   

  number                                        source_file     typeId  \
0  49281  https://en.wikipedia.org/wiki/Universal_Servic...  WIKIPEDIA   
1  49281  https://en.wikipedia.org/wiki/Universal_Servic...  WIKIPEDIA   
2  49281  https://en.wikipedia.org

In [83]:
wikipedia_data = wikipedia_extraction_state.get_all()

In [84]:
wikipedia_data['extractions'].keys()

dict_keys(['WIKI49281', 'WIKI58094', 'WIKI27275', 'WIKI85747'])

In [85]:
print(wikipedia_data['extractions']['WIKI27275'].keys())

dict_keys(['title', 'filename', 'number', 'source_file', 'typeId', 'sections', 'full_text', 'summary', 'citation', 'id', 'type', 'doc_id', 'source_document', 'source_dataset', 'topics', 'active', 'date', 'text', 'overview', 'overview_type', 'overview_pages', 'overview_citations', 'overview_cite_sources'])


In [86]:
print(wikipedia_data['extractions']['WIKI27275']['date'])

2025-06-08


In [87]:
print(wikipedia_data['extractions']['WIKI27275']['title'])

Internet in the United States


In [88]:
len(wikipedia_data['extractions']['WIKI27275']['overview'].split(' '))

5030

In [89]:
len(wikipedia_data['extractions']['WIKI27275']['full_text'].split(' '))

4646

In [91]:
ui_folder = Path('working_folder/nicegui')
ui_database_location = ui_folder.joinpath('ui_data.sqlite')
ui_conn = sqlite3.connect(ui_database_location)
ui_cursor = ui_conn.cursor()

In [92]:
ui_cursor.execute("""select * from papers""").fetchone()

('paper_a01_s01_001',
 'proj_alpha_01',
 'general',
 'The Landscape of Algorithmic Bias',
 '# Understanding Bias\\nAn overview of different types of bias in machine learning systems and their societal impact.',
 '2025-06-07 20:03:22',
 0)

In [93]:
def insert_new_research_paper(document_id, title, content, document_type, project_id, stream_id, cursor, conn):
    added_at = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    cursor.execute("""
                    INSERT OR REPLACE INTO papers (id, project_id, stream_id, title, content, added_at, is_new)
                    VALUES (?, ?, ?, ?, ?, ?, ?)
                    RETURNING id, project_id, title, added_at;
                    """, (document_id, project_id, stream_id, title, content, added_at, is_new))
    row = cursor.fetchone()
    conn.commit()
    return row

## Alter the table to add document_type

In [94]:
def insert_into_research_stream(id, project_id, subject, focus, project_created_at, cursor, conn):
    row = ui_cursor.execute("""SELECT * from research_streams
                     WHERE project_id = ? AND id = ? AND subject = ? AND focus = ?""",
                  (project_id, id, subject, focus)).fetchone()
    if len(row) == 0:
        cursor.execute("""
                      INSERT OR REPLACE INTO research_streams (id, project_id, subject, focus, created_at)
                      VALUES (?, ?, ?, ?, ?)
                      RETURNING id, project_id, subject, focus, created_at;
                      """,
                      (id, project_id, subject, focus, project_created_at)
                      )
        row = cursor.fetchone()
        conn.commit()
    return row

In [95]:
stream_id

'rural broadband'

In [96]:
id = wikipedia_data['extractions']['WIKI27275']['id'] + run_timestamp
title = 'EXT_' + wikipedia_data['extractions']['WIKI27275']['title']
content = wikipedia_data['extractions']['WIKI27275']['overview']
project_id = "proj_alpha_01"
project_created_at = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
added_at = project_created_at
stream_id = 'rural broadband'
stream_subject_id = 'rural broadband'
is_new = 1
subject = subject_matter
focus = focus

for _document_id, _extractions in wikipedia_data['extractions'].items():
    title = 'EXT_' + _extractions['title']
    content = _extractions['overview']
    added_at = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    insert_new_research_paper(_document_id, title, content, 'extraction', project_id, stream_id, ui_cursor, ui_conn)


In [98]:
for _document_id, _extractions in crs_data['extractions'].items():
    title = 'EXT_' + _extractions['title']
    content = _extractions['overview']
    added_at = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    insert_new_research_paper(_document_id, title, content, 'extraction', project_id, stream_id, ui_cursor, ui_conn)

In [99]:
ui_cursor.execute("""
                  INSERT INTO research_streams (id, project_id, subject, focus, created_at)
                  VALUES (?, ?, ?, ?, ?)
                  RETURNING id, project_id, subject, focus, created_at;
                  """,
                  (stream_id, project_id, subject, focus, project_created_at)
                  )
row = ui_cursor.fetchone()
ui_conn.commit()

In [53]:
ui_cursor.execute("""
    INSERT OR REPLACE INTO papers (id, project_id, stream_id, title, content, added_at, is_new)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    RETURNING id, project_id, title, added_at;
""", (id, project_id, stream_id, title, content, added_at, is_new))
row = ui_cursor.fetchone()
ui_conn.commit()